## Import libraries

In [51]:
import io
import itertools

import numpy as np
import sklearn.metrics

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import matplotlib.pyplot as plt

## Load and preprocess data

In [52]:
train_data = np.load('dataset/Trousers & Jeans - All - Train.npz')
validation_data = np.load('dataset/Trousers & Jeans - All - Validation.npz')
test_data = np.load('dataset/Trousers & Jeans - All - Test.npz')

In [53]:
list(train_data.keys())

['images', 'labels', 'genders']

In [54]:
images_train, labels_train = train_data['images'], train_data['labels']
images_val, labels_val = validation_data['images'], validation_data['labels']
images_test, labels_test = test_data['images'], test_data['labels']

In [55]:
np.unique(labels_train)

array([0, 1, 2, 3])

In [56]:
images_train.shape

(4004, 120, 90, 3)

In [57]:
labels_train.shape

(4004,)

In [58]:
images_val.shape

(500, 120, 90, 3)

In [59]:
labels_val.shape

(500,)

In [60]:
images_test.shape

(500, 120, 90, 3)

In [61]:
labels_test.shape

(500,)

In [62]:
# standardize 
images_train = images_train/255.0
images_val = images_val/255.0
images_test = images_test/255.0

## Create the model!

### Define hyparameters

In [63]:
EPOCHS = 15
BATCH_SIZE = 64

In [64]:
HP_LAMBDA = hp.HParam('lambda', hp.Discrete([1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2, 0.1]))

In [65]:
METRIC = 'accuracy'

with tf.summary.create_file_writer(r'logs/Model 4 (Trousers&Jeans_All_L2)/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams = [HP_LAMBDA],
        metrics = [hp.Metric(METRIC, display_name='accuracy')]
    )

### Model architecture

In [66]:
def train_test_model(hparams, session_num):
    model = tf.keras.Sequential([
        # we choose the valueus for kernel_num, kernel_size, and dense from the best of the hyperparameter tuning we do in model 3
        tf.keras.layers.Conv2D(96, 7, activation = 'relu', input_shape = (120,90,3), kernel_regularizer = tf.keras.regularizers.L2(hparams[HP_LAMBDA])),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(96, 3, activation = 'relu', kernel_regularizer = tf.keras.regularizers.L2(hparams[HP_LAMBDA])),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation = 'relu', kernel_regularizer = tf.keras.regularizers.L2(hparams[HP_LAMBDA])),
        tf.keras.layers.Dense(4, activation='softmax')
    ])
    
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(name = 'sparse_categorical_crossentropy')
    model.compile(optimizer = 'adam', loss = loss_fn, metrics = ['accuracy','sparse_categorical_crossentropy'])
    
    log_dir = "Logs\\Model 4 (Trousers&Jeans_All_L2)\\fit\\" + f"run-{session_num}"
    
    # functions for creating confusion matrix
    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
        cm (array, shape = [n, n]): a confusion matrix of integer classes
        class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

        return figure

    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""

        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')

        # Closing the figure prevents it from being displayed directly inside the notebook.
        plt.close(figure)

        buf.seek(0)

        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)

        # Add the batch dimension
        image = tf.expand_dims(image, 0)

        return image

    # Define a file writer variable for logging purposes
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')

    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = model.predict(images_val)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(labels_val, test_pred)

        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=['Trousers Male','Jeans Male','Trousers Female','Jeans Female'])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
            
    # callbacks
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_sparse_categorical_crossentropy', patience = 2, restore_best_weights = True)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end = log_confusion_matrix)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq = 1, profile_batch = 0)
    
    model.fit(images_train,
              labels_train,
              epochs = EPOCHS,
              batch_size = BATCH_SIZE,
              callbacks = [tensorboard_callback, cm_callback, early_stopping],
              validation_data = (images_val, labels_val),
              verbose = 1)
    
    _, accuracy, _ = model.evaluate(images_val,labels_val)
    
    model.save(r'saved_models\Model 4 (Trousers&Jeans_All_L2)\Run-{}'.format(session_num))
    
    return accuracy

In [67]:
def run(log_dir, hparams, session_num):
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC,accuracy,step=1)

In [68]:
session_num = 1
for lambda_size in HP_LAMBDA.domain.values:
    hparams = {
        HP_LAMBDA : lambda_size
    }
    run_name = f'run-{session_num}'
    print('--- Starting Trial:',run_name)
    print({h.name: hparams[h] for h in hparams})
    run("Logs/Model 4 (Trousers&Jeans_All_L2)/hparam_tuning/" + run_name, hparams, session_num)

    session_num += 1

--- Starting Trial: run-1
{'lambda': 1e-05}
Epoch 1/15
63/63 [==============================] - 79s 1s/step - loss: 1.5289 - accuracy: 0.4743 - sparse_categorical_crossentropy: 1.5171 - val_loss: 1.0400 - val_accuracy: 0.5700 - val_sparse_categorical_crossentropy: 1.0286
Epoch 2/15
63/63 [==============================] - 75s 1s/step - loss: 0.9390 - accuracy: 0.6184 - sparse_categorical_crossentropy: 0.9278 - val_loss: 0.9707 - val_accuracy: 0.6000 - val_sparse_categorical_crossentropy: 0.9597
Epoch 3/15
63/63 [==============================] - 80s 1s/step - loss: 0.8322 - accuracy: 0.6611 - sparse_categorical_crossentropy: 0.8211 - val_loss: 0.8807 - val_accuracy: 0.6480 - val_sparse_categorical_crossentropy: 0.8696
Epoch 4/15
63/63 [==============================] - 76s 1s/step - loss: 0.7628 - accuracy: 0.6908 - sparse_categorical_crossentropy: 0.7515 - val_loss: 0.8778 - val_accuracy: 0.6500 - val_sparse_categorical_crossentropy: 0.8662
Epoch 5/15
63/63 [==========================

63/63 [==============================] - 83s 1s/step - loss: 0.6684 - accuracy: 0.7455 - sparse_categorical_crossentropy: 0.6240 - val_loss: 0.7968 - val_accuracy: 0.6960 - val_sparse_categorical_crossentropy: 0.7526
Epoch 8/15
63/63 [==============================] - 75s 1s/step - loss: 0.6406 - accuracy: 0.7537 - sparse_categorical_crossentropy: 0.5965 - val_loss: 0.8101 - val_accuracy: 0.7140 - val_sparse_categorical_crossentropy: 0.7662
Epoch 9/15
16/16 [==============================] - 2s 152ms/step - loss: 0.7968 - accuracy: 0.6960 - sparse_categorical_crossentropy: 0.7526
INFO:tensorflow:Assets written to: saved_models\Model 4 (Trousers&Jeans_All_L2)\Run-3\assets
--- Starting Trial: run-4
{'lambda': 0.0005}
Epoch 1/15
63/63 [==============================] - 75s 1s/step - loss: 1.9232 - accuracy: 0.4523 - sparse_categorical_crossentropy: 1.5593 - val_loss: 1.3380 - val_accuracy: 0.5560 - val_sparse_categorical_crossentropy: 1.0490
Epoch 2/15
63/63 [=============================

63/63 [==============================] - 81s 1s/step - loss: 0.9652 - accuracy: 0.6988 - sparse_categorical_crossentropy: 0.7454 - val_loss: 1.0133 - val_accuracy: 0.6640 - val_sparse_categorical_crossentropy: 0.8022
Epoch 8/15
63/63 [==============================] - 72s 1s/step - loss: 0.9202 - accuracy: 0.7123 - sparse_categorical_crossentropy: 0.7127 - val_loss: 1.0081 - val_accuracy: 0.6960 - val_sparse_categorical_crossentropy: 0.8006
Epoch 9/15
63/63 [==============================] - 76s 1s/step - loss: 0.9727 - accuracy: 0.6828 - sparse_categorical_crossentropy: 0.7569 - val_loss: 1.0896 - val_accuracy: 0.6400 - val_sparse_categorical_crossentropy: 0.8685
Epoch 10/15
16/16 [==============================] - 2s 150ms/step - loss: 1.0081 - accuracy: 0.6960 - sparse_categorical_crossentropy: 0.8006
INFO:tensorflow:Assets written to: saved_models\Model 4 (Trousers&Jeans_All_L2)\Run-6\assets
--- Starting Trial: run-7
{'lambda': 0.01}
Epoch 1/15
63/63 [==============================